In [3]:
import numpy as np
from skimage import io, transform, img_as_float
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from scipy.ndimage import center_of_mass, shift

def get_asymmetry(image_path: str) -> dict:
    original_image = io.imread(image_path)

    # Ensure the image is binary
    if len(original_image.shape) == 3:  # if the image is RGB
        gray_image = rgb2gray(original_image)
        thresh = threshold_otsu(gray_image)
        binary_image = gray_image > thresh
    else:
        binary_image = original_image > 0  # Assuming image is already in grayscale

    # Calculate the center of mass
    com = center_of_mass(binary_image)

    # Calculate the shift needed to bring the center of mass to the center of the image
    shift_needed = np.array(binary_image.shape) / 2.0 - np.array(com)

    # Apply the shift to the binary image
    shifted_image = shift(binary_image, shift_needed)

    # Convert shifted binary image to float for rotation
    shifted_image_float = img_as_float(shifted_image)

    # Initialize the dictionary to store asymmetry values
    asymmetry_results = {}

    # Function to measure asymmetry
    def measure_asymmetry(binary_img):
        # Flip the image along both axes
        vertical_flip = np.flip(binary_img, axis=0)
        horizontal_flip = np.flip(binary_img, axis=1)

        # Compute the non-overlapping parts using logical XOR
        vertical_asymmetry = np.sum(np.logical_xor(binary_img, vertical_flip))
        horizontal_asymmetry = np.sum(np.logical_xor(binary_img, horizontal_flip))

        return vertical_asymmetry, horizontal_asymmetry

    # Rotate image from 0 to 90 degrees in 5 degree increments and measure asymmetry
    for angle in range(0, 91, 5):
        # Rotate the float image
        rotated_image_float = transform.rotate(shifted_image_float, angle, resize=True, center=None, order=1, mode='constant', cval=0, clip=True, preserve_range=True)
        
        # Convert rotated image back to binary
        rotated_image_binary = rotated_image_float > 0.5

        # Measure and record asymmetry for the binary rotated image
        asymmetry_results[angle] = measure_asymmetry(rotated_image_binary)

    return asymmetry_results

In [4]:
import os

asymmetry_data = {}  # Dictionary to store all results

path = "group_A_masks/masks/mask"

for i in range(1, 6):
    directory = f"{path}{i}"
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            asymmetry_values = get_asymmetry(f)
            print(f"{f}: {asymmetry_values}")
            asymmetry_data[f] = asymmetry_values

group_A_masks/masks/mask1/PAT_1109_437_254_mask.png: {0: (24782, 26406), 5: (18362, 17754), 10: (21544, 20250), 15: (29366, 28402), 20: (39760, 39622), 25: (50596, 51292), 30: (60074, 60470), 35: (67816, 67938), 40: (73658, 73430), 45: (76878, 76858), 50: (78338, 78426), 55: (77736, 77790), 60: (75160, 75172), 65: (70292, 70476), 70: (63716, 63760), 75: (55196, 55306), 80: (45842, 45686), 85: (36736, 36076), 90: (26406, 24782)}
group_A_masks/masks/mask1/PAT_1255_876_939_mask.png: {0: (113254, 122798), 5: (90592, 97174), 10: (78056, 69612), 15: (64662, 45102), 20: (64548, 50914), 25: (76788, 78222), 30: (99740, 103836), 35: (126362, 124254), 40: (148790, 145582), 45: (171278, 171406), 50: (187072, 188696), 55: (196306, 198276), 60: (201224, 202152), 65: (203404, 201064), 70: (201790, 195616), 75: (193334, 186202), 80: (174512, 171132), 85: (149822, 143454), 90: (122798, 113254)}
group_A_masks/masks/mask1/PAT_1210_738_915_mask.png: {0: (48540, 47884), 5: (46946, 43478), 10: (49088, 38048

In [5]:
import json

# Convert NumPy integers to Python integers before writing to JSON
for key, value in asymmetry_data.items():
    for angle, (vertical, horizontal) in value.items():
        # Convert NumPy int64 to Python int
        value[angle] = [int(vertical), int(horizontal)]

# Save results to JSON file
os.makedirs('raw_data', exist_ok=True)  # Ensure the directory exists
with open('raw_data/asymmetry.json', 'w') as outfile:
    # Use default=str in json.dump to handle other potential serialization issues
    json.dump(asymmetry_data, outfile, indent=4, default=str)
